<a href="https://colab.research.google.com/github/VVerdenko/oriented-2-trees/blob/main/2_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D1%8C%D1%8F_%D1%87%D0%B0%D1%81%D1%82%D0%B8%D1%87%D0%BD%D0%BE_%D0%BE%D1%80%D0%B8%D0%B5%D0%BD%D1%82%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5(%D0%B2%D1%8B%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%BE_%D0%BF%D0%BE_%D0%B2%D0%B5%D1%80%D1%88%D0%B8%D0%BD%D0%B5_%D0%B2_%D0%BA%D0%B0%D0%B6%D0%B4%D0%BE%D0%B9_%D1%8F%D1%87%D0%B5%D0%B9%D0%BA%D0%B5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
M_1 = [0] * 100
N_1 = [0] * 100
M = [0] * 100
N = [0] * 100

M_S = [0] * 100       # ряды в которые возводим экспоненту для нахождения M и M + 2N соответсвенно
N_S = [0] * 100

e = [0] * 100   # просто M(x) + 2*N(x) для удобства

s_1 = [0] * 100
s_0 = [0] * 100

s_0_Z = [0] * 100

t = [0] * 100

sp = []     # sp[n][k] хранит список разбиений числа n на k слогаемых, необходимо чтобы возводить экспоненту в степень с помощью ряда Тейлора
f = []      # массив факториалов

In [2]:
def find_f(max_n):
  f.append(1)
  for i in range(1, max_n+1):
    f.append(f[i-1] * i)

f = []
find_f(20)              # необходимо посчитать достаточно много факториалов и sp, прежде чем считать ряды

In [15]:
def find_sp(max_n):
  sp.append([[[]]])
  for i in range(1, max_n + 1):
    sp.append([[[]]])
    for j in range(1, i + 1):
      sp[i].append([[]])
    sp[i][1] = [[i]]

  # разбиваем n на k + 1 слогаемых, припсывая i в конец разбияения n - i на k слогаемых

  for n in range(2, max_n + 1):
    for i in range(1, n):
      for k in range(1, n - i + 1):

        x = sp[n - i][k][:]
        for j in range(len(x)):
          x[j] = x[j] + [i]
        if sp[n][k + 1] == [[]]:
          sp[n][k + 1] = x
        else:
          sp[n][k + 1] += x


sp = []
find_sp(20)

In [4]:
def find_M_1(n):
  sum = 0
  n -= 1
  if n % 2 == 0:
      k = n // 2
      sum += e[k]
  M_1[n + 1] = sum
  return sum

In [8]:
def find_N_1(n):
  sum = 0
  n -= 1
  # Сперва считаем коэффициент Z(A2 - S2)
  # это s1^2
  for i in range (1, n):
    sum += e[i] * e[n - i]
  sum += 2 * e[n]
  # это -s2
  if n % 2 == 0:
    k = n // 2
    sum -= e[k]
  sum = 3 * sum / 2

  if n % 2 == 0:
    k = n // 2
    sum += e[k]


  N_1[n + 1] = sum
  return sum

In [6]:
def find_M(n):
  sum = 0
  # Сперва посчитаем ряд в который возведем экспоненту
  for k in range(1, n + 1):
    if n % k == 0:
      q = n // k
      sum += M_1[q] / k
    if n % (2 * k) == 0:
      q = n // (2 * k)
      sum += N_1[q] / k
  M_S[n] = sum

  sum = 0
  for k in range(1, n + 1):      # номер слогаемого в ряде Тейлора
    y = 0
    for u in sp[n][k]:          # какие степени взять от каждой скобки, чтобы за к скобок получить н
      x = 1
      for i in u:
        x *= M_S[i]
      y += x
    sum += y / f[k]


  M[n] = sum
  return sum

In [7]:
def find_N(n):
  sum = 0
  # Сперва посчитаем ряд в который возведем экспоненту
  for k in range(1, n + 1):
    if n % k == 0:
      q = n // k
      sum += M_1[q] / k
      sum += 2 * N_1[q] / k
  N_S[n] = sum

  sum = 0
  for k in range(1, n + 1):      # номер слогаемого в ряде Тейлора
    y = 0
    for u in sp[n][k]:          # какие степени взять от каждой скобки, чтобы за к скобок получить н
      x = 1
      for i in u:
        x *= N_S[i]
      y += x
    sum += y / f[k]

  N[n] = (sum - M[n]) / 2
  return sum

In [9]:
def find_s_1(n):
  sum = 0
  for i in range(n):
    sum += M_1[i] * M[n - i]
  sum += M_1[n]
  s_1[n] = sum

In [11]:
def find_s_0(n):
  sum = 0
  n -= 1
  # Сперва считаем коэффициент Z(A2 - S2)
  # это s1^2
  for i in range (1, n):
    sum += e[i] * e[n - i]
  sum += 2 * e[n]
  # это -s2
  if n % 2 == 0:
    k = n // 2
    sum -= e[k]
  s_0_Z[n] = sum / 2

  sum = 0
  for i in range(n):
    sum += s_0_Z[i] * e[n - i]
  sum += s_0_Z[n]

  for i in range(n):
    if i % 2 == 0:
      k = i // 2
      sum += e[k] * N[n - i]
  sum += N[n]

  s_0[n + 1] = sum

In [13]:
def main(max_n):
  M[1] = 1
  N[1] = 1
  M_1[1] = 1
  N_1[1] = 1

  e[1] = 3

  s_1[1] = 1
  s_0[1] = 0

  M_S[1] = 1
  N_S[1] = 3

  for i in range(2, max_n + 1):
    find_M_1(i)
    find_N_1(i)
    find_M(i)
    find_N(i)
    M_1[i] = round(M_1[i])
    N_1[i] = round(N_1[i])
    M[i] = round(M[i])
    N[i] = round(N[i])

    e[i] = M[i] + 2 * N[i]

  for i in range(2, max_n + 1):
    find_s_1(i)
    find_s_0(i)
    s_0[i] = round(s_0[i])
    s_1[i] = round(s_1[i])

  for i in range(1, max_n + 1):
    t[i] = N[i] + M[i] - 2 * s_0[i] - s_1[i]

main(17)
print("M_1:", M_1)
print("N_1:", N_1)
print("M:  ", M)
print("N:  ", N)
print("s_0:", s_0)
print("s_1:", s_1)

M_1: [0, 1, 0, 3, 0, 24, 0, 235, 0, 2649, 0, 32196, 0, 412576, 0, 5483463, 0, 74907651, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
N_1: [0, 1, 9, 84, 921, 10914, 137349, 1800940, 24349230, 337002249, 4751319246, 67998098442, 985259083545, 14425004311264, 213071443745517, 3171381844939998, 47518046125278896, 716152964777946240, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
M:   [0, 1, 2, 5, 15, 42, 142, 431, 1569, 4985, 18852, 61637, 239631, 799167, 3166494, 10714553, 43066185, 147347309, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [14]:
print(t)

[0, 1, 4, 21, 168, 1566, 16720, 190693, 2289774, 28513359, 365510528, 4796000656, 64160402181, 872447068287, 12030114471609, 167896397782184, 2368058522401474, 33711610641473076, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
